### **Importing Libraries**


In [7]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from networkx.algorithms.community.centrality import girvan_newman
from operator import mul
import networkx as nx
import os
import sys
import community.community_louvain as community
import matplotlib.image as mpimg
import matplotlib.colors as mpcolors
import matplotlib.cm as mpcm
import random

### **Loading DataSet**

In [8]:
data_url = "./Dataset/Cit-HepPh.txt"
df_data_1 = pd.read_csv(data_url, sep='\t', skiprows=4, names=['FromNodeId', 'ToNodeId'], dtype={'FromNodeId': int, 'ToNodeId': int})

### **Loading Time of Release**

In [37]:
data_url = "./Dataset/cit-HepPh-dates.txt"
df_data_2 = pd.read_csv(data_url, sep='\t', skiprows=1, names=['NodeId', 'Date'], dtype={'NodeId': str, 'Date': str})
df_data_2['Date'] = pd.to_datetime(df_data_2['Date'])
graphs_by_date = {}
# Filter entries with '11' at the beginning of the 'NodeId' column
df_data_2 = df_data_2[~df_data_2['NodeId'].str.startswith('11')]
# Filter entries with leading 0's of the 'NodeId' column
df_data_2['NodeId'] = df_data_2['NodeId'].astype(str).str.lstrip('0')
df_data_2['NodeId'] = df_data_2['NodeId'].astype(int)
df_data_2 = df_data_2[df_data_2['Date'].dt.year <= 1993]
i = 0
# unnodes = df_data_2['NodeId']
# for nodes in unnodes:
#     print(nodes)
#     i += 1
# print(i)


### **Merging Both DataSet**

In [38]:
df_merged = pd.merge(df_data_1, df_data_2, how='inner', left_on='FromNodeId', right_on='NodeId')
df_merged['Date'] = pd.to_datetime(df_merged['Date'])
# Filter out rows where 'ToNodeId' is not present in 'NodeId' column of df_data_2
df_merged = df_merged[df_merged['ToNodeId'].isin(df_data_2['NodeId'])]
unnodes = df_merged['FromNodeId'].unique()
i = 0
for nodes in unnodes:
    # print(node)
    i += 1
print(i)

1037


### **Creation of Graph**

In [5]:

# Construct the directed graph
G_lat = nx.from_pandas_edgelist(df_merged, 'FromNodeId', 'ToNodeId', create_using=nx.DiGraph())

print("Number of nodes:", len(G_lat.nodes()))
print("Number of edges:", len(G_lat.edges()))
print(nx.density(G_lat))


Number of nodes: 30501
Number of edges: 347268
0.0003732938613676198


### **Yearly Analysis**

In [34]:
df_merged['Year'] = df_merged['Date'].dt.year


# Counter to keep track of processed years
flag = 0

for year in range(1992, 1994):

    month = 1
    while(month < 13):
        # Calculate the end month considering the three-month gap
        end_month = month + 2
        end_year = year
        if end_month > 12:
            end_month -= 12
            end_year += 1

        # Filter the data for the current three-month period
        filtered_data = df_merged[ (df_merged['Date'].dt.year < year) | ( (df_merged['Date'].dt.year == year) & (df_merged['Date'].dt.month >= month) & (df_merged['Date'].dt.month <= end_month))]

        if not filtered_data.empty:
            G = nx.from_pandas_edgelist(filtered_data, 'FromNodeId', 'ToNodeId', create_using=nx.DiGraph())
            inbuilt_communities_generator = girvan_newman(G, most_valuable_edge=None)
            inbuilt_communities = [c for c in next(inbuilt_communities_generator)]
            
            # Process your communities or do any other desired operations
            
            # flag += 1
            # # Assign random colors to each detected community ensuring uniqueness
            # community_colors = {}
            # used_colors = set()  # Set to store used colors
            # for i, community in enumerate(inbuilt_communities):
            #     color = "#{:06x}".format(random.randint(0, 0xFFFFFF))  # Generate a random hex color
            #     community_colors[i] = color
            #     used_colors.add(color)
            #     for node in community:
            #         G.nodes[node]['community'] = i
            #         G.nodes[node]['color'] = color

            # # Export the graph to GraphML format
            # nx.write_graphml(G, f"graph_{flag}_3months_{year}-{month}-to-{end_year}-{end_month}.graphml")
        month = end_month + 1
